# Modeling Relationships

To work with related objects we have two approaches:

**Using References (Normalization):** We have separate collections for storing objects. In relational databases we have this concept of relationship wich enforces data itegrity. In no sql databases we don´t have a relationship

**Using embbeded documents (Denormalization):** Instead of having a separte collection of authors we can embed an author document inside of a course document.

In [ ]:
// Using References Normalization -- CONSISTENCY
let author = {
    name: 'Name',
}

let course = {
    author: 'id' // this id is a reference to an author in authors collection
}

// Using Embedded Documents (Denormalization) -- PERFORMANCE
let course = {
    author: {
        name: 'Name'
    }
}

- What approach to use depends of the aplications and its query requirements.
- Basically we need to do a trade off between query performance vs consistency.
- With the first approach we have consitency because if we need for example modify the name property of author, there is a single place that we need to modify and all courses that are reference in that author will immediatly see the update. Hiwever, everytime that we query a course, we need to do an extra query to load the related author.

- With the second approach we can load de we can the course object and its author using a single query. However with this approach, we need make changes in multiple documents  

**Hybrid:** For example, we have an author object with 50 properties, we dont want to duplicate those properties inside every course in the database (Denormalization approach). We can have a separate collection of authors but instead of using a reference (Normalization approach), we can embed an author document inside a course document but not the complte representation of that author, perhaps only te name propertie.

With this approach we can quickly read a course object along its author so we can optimize the query performance, but we dont have to store all the properties about an author inside the course document. This approach is useful if we want to have a snapshot of our data at a point in time.

For example in an e-comerce application there would have collections like orders, products, shopping carts nad so on. In each order we need to store a snapshot of a product because we want to know the price of that product at a given point in time.

In [ ]:
//Hybrid
let author = {
    name: 'Name'
    // 50 other properties
}

let course = {
    author:{
        id: 'ref',
        name; 'Name'
    }
}

# Referencing Documents

- We will reference a document in another document. We create a filed called *population.js* in wich we have two models: Author and Course.

In [ ]:
//population.js
//------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to MongoDB...'))
    .catch(err => console.error('Could not connect to MongoDB...', err))

const Author = mongoose.model('Author', new mongoose.Schema({
    name:String,
    bio: String,
    website: String
}));

const Course = mongoose.model('Course', new mongoose.Schema({
    name:String
}));

async function createAuthor(name, bio, website){
    let author = new Author({
        name,
        bio,
        website
    });
    author = await author.save();
    console.log(author);
}

async function createCourse(name, author){
    let course = new Course({
        name: name,
        author: author
    });
    course = await course.save();
    console.log(course);
}

async function listCourses(){
    const courses = await Course
        .find()
        .select('name author');
    
    console.log(courses);
    
}

// createAuthor('Sebastian', 'MyBio', 'My Website');

createCourse('My Course', '62f16ce495d56688b86b79d9');

- We first create a new author:
    
    **output:** Connected to MongoDB...
    {
      name: 'Sebastian',
      bio: 'MyBio',
      website: 'My Website',
      _id: new ObjectId("62f16ce495d56688b86b79d9"),
      __v: 0
    }

- Then we create a new course referencing the created author by id:  

    **output:**
    Connected to MongoDB...
    {
      name: 'My Course',
      _id: new ObjectId("62f16e44094b19b6e815d80b"),
      __v: 0
    }
    
We can see that created course does not have the **author** propertie. The reason is that when we define the *Course* model we did not add the *author* propertie. When we save the object, only the properties that we define in the model will be persistent in the database.

We add the author property to the Course model. Its type will be **mongoose.Schema.Types.ObjectId** and **ref** to Author 
In this sense, this *author* propertie would store an object ID that references an *Author* document. However we dont really have a relationship here because we can store a course with an invalid author and mongo does not complain about that.


In [ ]:
    .
    .
    .
const Course = mongoose.model('Course', new mongoose.Schema({
    name:String,
    author: {
        type: mongoose.Schema.Types.ObjectId,
        ref: 'Author'
    }
}));
    .
    .
    .
    
//console
//C:\Users\sebastian\Desktop\nodejs\09_Mongoose- Relationships between Connected Data>node population.js

    
//output
// Connected to MongoDB...
// {
//   name: 'My Course',
//   author: new ObjectId("62f16ce495d56688b86b79d9"),
//   _id: new ObjectId("62f1727130ce4880049647da"),
//   __v: 0
// }

# Population

- The output of **listCourses** is:
    
    Connected to MongoDB...
    [
      { _id: new ObjectId("62f16e44094b19b6e815d80b"), name: 'My Course' },
      {
        _id: new ObjectId("62f1727130ce4880049647da"),
        name: 'My Course',
        author: new ObjectId("62f16ce495d56688b86b79d9")
      }
    ]
    
 We can see that in the second course the author property just show the id. In a real world application we want to load this author document and display its name. For this we use the **populate** method, with **author** as a neame of the target property.

In [ ]:
    .
    .
    .
async function listCourses(){
    const courses = await Course
        .find()
        .populate('author')
        .select('name author');
    
    console.log(courses);
    
}

// createAuthor('Sebastian', 'MyBio', 'My Website');
// createCourse('My Course', '62f16ce495d56688b86b79d9');
listCourses();



We can see that after using **populate** method, output shows the author properties:

    Connected to MongoDB...
    [
      { _id: new ObjectId("62f16e44094b19b6e815d80b"), name: 'My Course' },
      {
        _id: new ObjectId("62f1727130ce4880049647da"),
        name: 'My Course',
        author: {
          _id: new ObjectId("62f16ce495d56688b86b79d9"),
          name: 'Sebastian',
          bio: 'MyBio',
          website: 'My Website',
          __v: 0
        }
      }
    ]

We can especify the properties we want to include/exclude in the second argument of **populate**. To show just the name propertie, we add 'name', and, to exclude the _id property we use '-_id' as a second argument

In [ ]:
    .
    .
    .
async function listCourses(){
    const courses = await Course
        .find()
        .populate('author', 'name -_id')
        .select('name author');
    
    console.log(courses);
    
}

// createAuthor('Sebastian', 'MyBio', 'My Website');
// createCourse('My Course', '62f16ce495d56688b86b79d9');
listCourses();


// output
//------------------------------------------------------------------------
// Connected to MongoDB...
// [
//   { _id: new ObjectId("62f16e44094b19b6e815d80b"), name: 'My Course' },
//   {
//     _id: new ObjectId("62f1727130ce4880049647da"),
//     name: 'My Course',
//     author: {
//       name: 'Sebastian'
//     }
//   }
// ]

# Embedding Documents

- In this lecture we are going to emmbed an **author** document directly iniside of a course document. To do this, we first define an **authorSchema** outside model definition. Then, in the **Course** model we define author property as *authorSchema* type.

- We can see that author property of the created course is an object with a name and id; Ths is an embedded or subdocument. Subdocuments are like documents; most of features that ae available to normal documents, are also available in subdocuments. 

In [ ]:
//embedding.js
//------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to MongoDB...'))
    .catch(err => console.error('Could not connect to MongoDB...', err))

const authorSchema = new mongoose.Schema({
    name:String,
    bio: String,
    website: String
});

const Author = mongoose.model('Author', authorSchema);

const Course = mongoose.model('Course', new mongoose.Schema({
    name:String,
    author: authorSchema
}));

async function createAuthor(name, bio, website){
    let author = new Author({
        name,
        bio,
        website
    });
    author = await author.save();
    console.log(author);
}

async function createCourse(name, author){
    let course = new Course({
        name: name,
        author: author
    });
    course = await course.save();
    console.log(course);
}

async function listCourses(){
    const courses = await Course
        .find()
        .populate('author', 'name')
        .select('name author');
    
    console.log(courses);
    
}

createCourse('My Course', new Author({name: 'Sebastian'}));



//output
// Connected to MongoDB...
// {
//   name: 'My Course',
//   author: { name: 'Sebastian', _id: new ObjectId("62f2b29394e3593ead5b60c0") },
//   _id: new ObjectId("62f2b29394e3593ead5b60c1"),
//   __v: 0
// }

- However, the subdocuments can only be saved in the context of their parent. For example, if we want to update an author we need to specify the id of the parent course.


In [ ]:
    .
    .
    .
async function updateAuthor(courseID){
    const course = await Course.findById(courseID);
    course.author.name = 'Sebastian Gil';
    course.save();
}

updateAuthor('62f2b29394e3593ead5b60c1')


//output
//------------------------------------------------
// Connected to MongoDB...
// {
//   _id: new ObjectId("62f2b29394e3593ead5b60c1"),
//   name: 'My Course',
//   author: {
//     name: 'Sebastian Gil',
//     _id: new ObjectId("62f2b29394e3593ead5b60c0")
//   },
//   __v: 0
// }

- We can also update a subdocument directly:

In [ ]:
    .
    .
    .
async function updateAuthor(courseId){
    const course = await Course.updateOne({_id: courseId}, {
        $set: {
            'author.name': 'Julian'
        }
    });
}

updateAuthor('62f2b29394e3593ead5b60c1')


//output
//------------------------------------------------
// Connected to MongoDB...
// {
//   _id: new ObjectId("62f2b29394e3593ead5b60c1"),
//   name: 'My Course',
//   author: {
//     name: 'Julian',
//     _id: new ObjectId("62f2b29394e3593ead5b60c0")
//   },
//   __v: 0
// }

- To remove a subdocument property or whole subdocument we use **unset** opeartor.

In [ ]:
    .
    .
    .
async function deleteAuthor(courseId){
    const course = await Course.updateOne({_id: courseId}, {
        $unset: {
//             'author.name': '' // to remove name property
            'author': '' // to remove whole subdocument
        }
    });
}

deleteAuthor('62f2b29394e3593ead5b60c1')


//output
//------------------------------------------------
// Connected to MongoDB...
// {
//   _id: new ObjectId("62f2b29394e3593ead5b60c1"),
//   name: 'My Course',
//   __v: 0
// }

# Using an Array of Subdocuments

In [ ]:
//population.js
//------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to MongoDB...'))
    .catch(err => console.error('Could not connect to MongoDB...', err))

const authorSchema = new mongoose.Schema({
    name:String,
    bio: String,
    website: String
});

const Author = mongoose.model('Author', authorSchema);

const Course = mongoose.model('Course', new mongoose.Schema({
    name:String,
    authors: [authorSchema]
}));

async function createAuthor(name, bio, website){
    let author = new Author({
        name,
        bio,
        website
    });
    author = await author.save();
    console.log(author);
}

async function createCourse(name, authors){
    let course = new Course({
        name,
        authors
    });
    course = await course.save();
    console.log(course);
}

createCourse('Course with several authors', [
    new Author({name: 'Sebastian'}),
    new Author({name: 'Julian'})
]);

//output
//---------------------------
// Connected to MongoDB...
// {
//   name: 'Course with several authors',
//   authors: [
//     {
//       name: 'Sebastian',
//       _id: new ObjectId("62f2c1b79ec2b1986249d17f")
//     },
//     { name: 'Julian', _id: new ObjectId("62f2c1b79ec2b1986249d180") }
//   ],
//   _id: new ObjectId("62f2c1b89ec2b1986249d181"),
//   __v: 0
// }

We can also add authors after course creation

In [ ]:
    .
    .
    .
async function addAuthor(courseId, author){
    const course = await Course.findById(courseId);
    course.authors.push(author);
    course.save();
}

addAuthor('62f2c1b89ec2b1986249d181', new Author({name: 'New Author'}));

//---------------------------
// Connected to MongoDB...
// {
//   name: 'Course with several authors',
//   authors: [
//     {
//       name: 'Sebastian',
//       _id: new ObjectId("62f2c1b79ec2b1986249d17f")
//     },
//     { name: 'Julian', _id: new ObjectId("62f2c1b79ec2b1986249d180") }
//     { name: 'New Author', _id: new ObjectId("62f2c3b644b51c630c8d64d2
//") }
//   ],
//   _id: new ObjectId("62f2c1b89ec2b1986249d181"),
//   __v: 0
// }


Removing author:

In [ ]:
async function removeAuthor(courseId, authorId){
    const course = await Course.findById(courseId);
    const author = course.author.id(authorId);
    author.remove();
    course.save();
}

removeAuthor('62f2c1b89ec2b1986249d181', '62f2c3b644b51c630c8d64d2')

//output
//---------------------------
// Connected to MongoDB...
// {
//   name: 'Course with several authors',
//   authors: [
//     {
//       name: 'Sebastian',
//       _id: new ObjectId("62f2c1b79ec2b1986249d17f")
//     },
//     { name: 'Julian', _id: new ObjectId("62f2c1b79ec2b1986249d180") }
//   ],
//   _id: new ObjectId("62f2c1b89ec2b1986249d181"),
//   __v: 2
// }

# Project - Build the Dishes API

We create dishes api for the **Restaurant** project. For each created dish we have a menu as subdocument

# Project - Build the Orders API

We create orders api for the **Restaurant** project. For each created order we have a customer and dish as subdocuments.

However, we could have a problem. For example, if we create a new order of a dish that contains a product with limited quantity, we need discount the quantity of that product and update the database of inventary:

         .
         .
         .
    order = await order.save();

    product.numberInStock--;
    product.save();
         .
         .
         .

It is posible that after saving order (**order.save()**), something goes wrong, either a server creashes or connection to mongodb drops, so perhaps the second operation (**product.save()**) would not complete.

To solve this problem we can use **transactions**. With this we ensure that both operations will update the state of the data in the database or none of them will be applied. In relational databases we have this concept of transactions but for older versions of mongodb it does not exist. The technique **Two-Phase commit** is an advance topic related to mongo and can be used to solve the problem. 


# MongoDB Atlas

MongoDB Atlas is a fully-managed cloud database that handles all the complexity of deploying, managing, and healing your deployments on the cloud service provider of your choice (AWS , Azure, and GCP). MongoDB Atlas is the best way to deploy, run, and scale MongoDB in the cloud.

**To begin using MongoDB Atlas we need to do the following**:

1. Create a MongoDB cloud account.
2. Create a MongoDB Atlas cluster (free option for this case).
3. Configure network access and create a cluster user.
    - ip address: 0.0.0.0/0 (Allow access from anywhere)
    - Username: whatever
    - Password: whatever
4. Connect to the cluster.
    - connection method: Connect your application
    - copy the connection string (mongodb+srv://<username>:<password>@cluster0.reo2xyq.mongodb.net/?retryWrites=true&w=majority)
    - Go to application code an paste the connection string replacing *<password>* with the password in step 3.


In [ ]:
// connecting to MongoDB Atlas Cluster
mongoose.connect('mongodb+srv://<username>:<password>@cluster0.reo2xyq.mongodb.net/?retryWrites=true&w=majority')
    .then(() => {console.log('Connected to mongoDB...')})
    .catch(err => {console.error('Could not connect to mongoDB...')});


* Now, we can work with cloud database in the same way as we did with the local database. In Postman the query string is the same for cloud database; for example: (http://localhost:3000/api/...).

* The cloud database cretaed has the **replica set** feature.

* **Replication in MongoDB**: A replica set in MongoDB is a group of mongod processes that maintain the same data set. Replica sets provide redundancy and high availability, and are the basis for all production deployments


#  Transactions

- In some relational databases we have the concept of transactions, wich means that a group of opeartions should be performed as a unit, i.e, all completed or all roll back to initial state. In recent versions of mongoose this concept has been implemented.

- In Mongoose, transactions are only available in **replica set environment**. If we dont have replica set, we can convert our local databse to one.

- As we can see, we put the operations in the *try* block. If the operations complete succesfully, transaction is commited; if not transaction is aborted and all roll back to initial state.

In [ ]:
            .
            .
            .
const mongoose = require('mongoose');

const session = await mongoose.startSession();
    try{
        session.startTransaction();
//         First Operation...
//         Second Operation...
//             .
//             .
//             .
        await session.commitTransaction();        
    }
    catch(ex){
        await session.abortTransaction();
        console.log(ex);
    }
    session.endSession();
            .
            .
            .

Using transaction with **Orders API**

In [ ]:
                .
                .
                .
router.post('/', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(400).send(error.details[0].message);
    
    const customer = await Customer.findById(req.body.customerId);
    if (!customer) return res.status(400).send('Invalid customer.');

    const dish = await Dish.findById(req.body.dishId);
    if (!dish) return res.status(400).send('Invalid dish.');
    
    let order = new Order({
        customer: {
            _id: customer._id,
            name: customer.name,
            phone: customer.phone
        },

        dish: {
            _id: dish._id,
            name: dish.name,
            price: dish.price
        },
    });

    dish.name = 'New name' // cnahnge in dishes collection

    const session = await mongoose.startSession();
    try{
        session.startTransaction();
        order = await order.save({session}); // save order
        await dish.save({session}); // save dish
        // res.send(order, {session});// generating error
        res.send(order);
        await session.commitTransaction();        
    }
    catch(ex){
        await session.abortTransaction();
        console.log(ex);
    }
    session.endSession();
});
                .
                .
                .

# Object ID

- When we crete an object in mongoDB, it sets the id to a long string like "62f535aad59da8b12fc07f6e". It contains 24 characters and every two chracateres represent a byte. First 4 bytes represent a **timestamp**. Next 3 bytes represent **machine identifier**. Next two bytes represent **process identifier**. 3 last bytes represnt **counter**. With these 12 bytes we can unique identify a documet in mongoDB.

- If we generate at the same timestamp, at the same machine, at the same process more than 16M (2^24) of docuemnts, counter wich have 3 bytes will overflow. So the objectID will be the same for two docuemnts. 

- Actually, ID is generated by **mongoDB Driver**, that means that we dont have to wait for mongoDB to generate new ID. That is why applications built on top of mongoDB are highly scalable.we can have several insatnces of mongoDB and we dont have to talk to a central place to get an unique identifier. Driver itself can generate almost unique ID using mentioned 12 bytes.

- Mongoose is an abstranction over **mongoDB Driver**. When we create a new nomgoDB object, mongoose talk to MongoDB Driver to generate a new ID.

- We can explictly generate an ID:

In [12]:
const mongoose = require('mongoose');
const id = new mongoose.Types.ObjectId();
console.log(id);

new ObjectId("62f6779f0a1499810f5c1f15")


undefined

In [14]:
console.log(id.getTimestamp()); // Timestamp of created ID

2022-08-12T15:54:07.000Z


undefined

In [15]:
var isValid = mongoose1.Types.ObjectId.isValid('1234'); //ID validation
console.log(isValid)

false


undefined

# Validating ObjectIDs

- We will implement id validation in our **Restaurant** Project each time we pass an id to create an order. The correct place to make this validation is in **validate function** of *Order* module.

- There is an npm package for adding support to validating object IDs in Joi. **npm i joi-objectid**

- require(joi-objectid) returns a function. so we need to call that function and pass a reference to Joi module (require(joi-objectid)(Joi)). The result of this is a function so we can set new Joi method as **Joi.objectId =  require(joi-objectid)(Joi)**

- In **validate** function we change *Joi.String()* to **Joi.objectID**. In this way the customerId and dishId will be validate as objectId instead of string. 

- In this way we no longer get unhandled promise rejection

In [ ]:
//Restaurant/models/order.js

const Joi = require('joi');
Joi.objectId = require('joi-objectid')(Joi);

            .
            .
            .
            
function validateOrder(order){
    const schema = Joi.object({
        customerId: Joi.objectId()
            .required(),
        dishId: Joi.objectId()
            .required()        
    });

    return schema.validate(order);
}

module.exports.Order = Order;
module.exports.validate = validateOrder;

# A better Implementation

- We need use Objectid validation in oder modules, like dish or customer modules. We dont want to redefine the object id method in every module.

- We load Joi.objectId in index.js and use it in **validateDish** function when we validate the menuId.

- In Dish mocule we create a new dish object, save it to the database ans then return it. In that implementation we reset that created dish after saving it to the database, but this is only for demonstrate that *save()* method returns a movie document, and then, talks to mongoDB Driver and sets the id. However we dont need to reset dish object in order to return that id to the client. So we remove the redefinition and change the dish from a variable to a constant. The same principle applies when we create a new customer ans a new menu.

In [ ]:
//Restaurant/router/dishes.js
                .
                .
                .
                
router.post('/', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(400).send(error.details[0].message);
        
    const menu = await Menu.findById(req.body.menuId);
    if (!menu) return res.status(400).send('Invalid menu.')

    const dish = new Dish({ // changed from let to const
        name: req.body.name,
        menu: {
            _id: menu._id,
            name: menu.name
        },
        price: req.body.price
    });

    await dish.save(); // Redefinition removed
    res.send(dish);
});
                .
                .
                .